In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator, MACCSkeys
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import xgboost as xgb

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 999

In [3]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  # 결측치 수정
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df['mol'] = train_df['mol'].apply(lambda x: Chem.AddHs(x))
  train_df['num_of_atoms'] = train_df['mol'].apply(lambda x: x.GetNumAtoms())
  train_df['num_of_heavy_atoms'] = train_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  test_df['mol'] = test_df['mol'].apply(lambda x: Chem.AddHs(x))
  test_df['num_of_atoms'] = test_df['mol'].apply(lambda x: x.GetNumAtoms())
  test_df['num_of_heavy_atoms'] = test_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  # 중복제거
  def canonize(mol):
    return Chem.MolToSmiles(Chem.MolFromSmiles(mol), isomericSmiles=True, canonical=True)

  canon_smile = []
  for molecule in train_df['SMILES']:
      canon_smile.append(canonize(molecule))
  
  train_df['canon_smiles'] = canon_smile
  
  ind = train_df.index[train_df['canon_smiles'].duplicated()]
  train_df = train_df.drop(ind)
  
  # MLM, HLM 100 이상 제거 코드 추가 예정
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  
  train_maccs = train_df["mol"].apply(lambda x: list(MACCSkeys.GenMACCSKeys(x).ToBitString()))
  train_maccs = np.stack(train_maccs, dtype=object)
  
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  test_maccs = test_df["mol"].apply(lambda x: list(MACCSkeys.GenMACCSKeys(x).ToBitString()))
  test_maccs = np.stack(test_maccs, dtype=object)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  
  train_features = np.append(np.append(origin_train_features, train_fps, axis=1), train_maccs, axis=1)
  test_features = np.append(np.append(origin_test_features, test_fps, axis=1), test_maccs, axis=1)
  target = train_df["MLM"].values
  
  return train_features, target, test_features
  

In [4]:
train_x, train_y, test_x = feature_engineer()

In [5]:
def create_model(param):
  return xgb.XGBRegressor(
    random_state=RANDOM_SEED,
    tree_method="gpu_hist",
    eval_metric="rmse",
    early_stopping_rounds=50,
    **param
  )

In [6]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [7]:
def objective_xgb(trial):
    param = {
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
        "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }

    if param["booster"] == "dart":
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
        param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=100)

[I 2023-09-01 17:11:57,798] A new study created in memory with name: xgb_tuning


[0]	validation_0-rmse:51.39494	validation_1-rmse:50.44044
[499]	validation_0-rmse:29.66932	validation_1-rmse:32.09180
Fold: 1/5 score = 1029.88362
[0]	validation_0-rmse:51.53237	validation_1-rmse:49.88245
[499]	validation_0-rmse:29.95179	validation_1-rmse:31.06475
Fold: 2/5 score = 965.01877
[0]	validation_0-rmse:51.01084	validation_1-rmse:52.00832
[499]	validation_0-rmse:29.61252	validation_1-rmse:32.78851
Fold: 3/5 score = 1075.08659
[0]	validation_0-rmse:50.81924	validation_1-rmse:52.73813
[499]	validation_0-rmse:29.72587	validation_1-rmse:32.47614
Fold: 4/5 score = 1054.69991
[0]	validation_0-rmse:51.28069	validation_1-rmse:50.93952
[499]	validation_0-rmse:29.97911	validation_1-rmse:31.13011


[I 2023-09-01 17:12:23,618] Trial 0 finished with value: 1018.754560793884 and parameters: {'booster': 'gbtree', 'lambda': 8.972050723983975e-08, 'alpha': 0.0013105438415021338, 'subsample': 0.18183273649667198, 'colsample_bytree': 0.39900311271371103, 'n_estimators': 500, 'max_depth': 6, 'min_child_weight': 7, 'eta': 0.0037944038609466855, 'gamma': 0.02087360741176864, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 1018.754560793884.


Fold: 5/5 score = 969.08392
[0]	validation_0-rmse:51.52324	validation_1-rmse:50.55915
[499]	validation_0-rmse:51.52226	validation_1-rmse:50.55823
Fold: 1/5 score = 2556.13496
[0]	validation_0-rmse:51.65957	validation_1-rmse:49.99981
[499]	validation_0-rmse:51.65859	validation_1-rmse:49.99889
Fold: 2/5 score = 2499.88932
[0]	validation_0-rmse:51.13149	validation_1-rmse:52.12513
[499]	validation_0-rmse:51.13052	validation_1-rmse:52.12419
Fold: 3/5 score = 2716.93116
[0]	validation_0-rmse:50.94226	validation_1-rmse:52.86080
[499]	validation_0-rmse:50.94130	validation_1-rmse:52.85985
Fold: 4/5 score = 2794.16347
[0]	validation_0-rmse:51.39918	validation_1-rmse:51.06110
[499]	validation_0-rmse:51.39821	validation_1-rmse:51.06013
Fold: 5/5 score = 2607.13708


[I 2023-09-01 17:27:48,539] Trial 1 finished with value: 2634.8511992595486 and parameters: {'booster': 'dart', 'lambda': 1.8500953639202822e-08, 'alpha': 0.18793183264405056, 'subsample': 0.46464459839419614, 'colsample_bytree': 0.7843919233053798, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 8, 'eta': 5.98025286065864e-08, 'gamma': 1.990042294039164e-05, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 4.545948976815658e-06, 'skip_drop': 0.6111537885501571}. Best is trial 0 with value: 1018.754560793884.


...

[I 2023-09-01 23:27:18,818] Trial 97 finished with value: 935.1081109060211 and parameters: {'booster': 'dart', 'lambda': 0.02896794878081653, 'alpha': 0.004613714323477511, 'subsample': 0.4553784752738193, 'colsample_bytree': 0.1950943560986113, 'n_estimators': 500, 'max_depth': 9, 'min_child_weight': 8, 'eta': 0.014170960576870506, 'gamma': 0.002857982570483273, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.2754511727182471e-06, 'skip_drop': 4.326482981586113e-08}. Best is trial 86 with value: 929.552645697805.


[0]	validation_0-rmse:50.04810	validation_1-rmse:49.25305
[144]	validation_0-rmse:21.09577	validation_1-rmse:31.47470
Fold: 1/5 score = 984.13185
[0]	validation_0-rmse:50.13497	validation_1-rmse:48.55625
[138]	validation_0-rmse:21.53565	validation_1-rmse:30.22681
Fold: 2/5 score = 904.51881


[I 2023-09-01 23:27:55,211] Trial 98 pruned. 


[0]	validation_0-rmse:51.34379	validation_1-rmse:50.40135
[499]	validation_0-rmse:24.08906	validation_1-rmse:31.22919
Fold: 1/5 score = 975.26256
[0]	validation_0-rmse:51.48185	validation_1-rmse:49.82744
[499]	validation_0-rmse:24.32269	validation_1-rmse:30.06740
Fold: 2/5 score = 904.04879
[0]	validation_0-rmse:50.95251	validation_1-rmse:51.95908
[499]	validation_0-rmse:23.97406	validation_1-rmse:31.66314
Fold: 3/5 score = 1002.55442
[0]	validation_0-rmse:50.76468	validation_1-rmse:52.69312
[499]	validation_0-rmse:24.25814	validation_1-rmse:31.42225
Fold: 4/5 score = 987.35776


[I 2023-09-01 23:40:31,045] Trial 99 pruned. 


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'booster': 'dart', 'lambda': 0.0005703799016250783, 'alpha': 0.00367023034708573, 'subsample': 0.4193233983885576, 'colsample_bytree': 0.2981574974790069, 'n_estimators': 500, 'max_depth': 8, 'min_child_weight': 8, 'eta': 0.012975774043582015, 'gamma': 0.005556218492117751, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 3.359580225438515e-07, 'skip_drop': 1.0398506127991463e-08}


In [11]:
best_param = {
  'booster': 'dart',
  'lambda': 0.0005703799016250783,
  'alpha': 0.00367023034708573,
  'subsample': 0.4193233983885576,
  'colsample_bytree': 0.2981574974790069,
  'n_estimators': 500,
  'max_depth': 8,
  'min_child_weight': 8,
  'eta': 0.012975774043582015,
  'gamma': 0.005556218492117751,
  'grow_policy': 'lossguide',
  'sample_type': 'uniform',
  'normalize_type': 'forest',
  'rate_drop': 3.359580225438515e-07,
  'skip_drop': 1.0398506127991463e-08
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

[0]	validation_0-rmse:51.12235	validation_1-rmse:50.20207
[499]	validation_0-rmse:21.54396	validation_1-rmse:30.82287
Fold: 1/5 score = 948.25135
[0]	validation_0-rmse:51.25663	validation_1-rmse:49.60928
[499]	validation_0-rmse:21.99393	validation_1-rmse:29.84328
Fold: 2/5 score = 890.59358
[0]	validation_0-rmse:50.72389	validation_1-rmse:51.74804
[499]	validation_0-rmse:21.79935	validation_1-rmse:31.23677
Fold: 3/5 score = 974.59757
[0]	validation_0-rmse:50.54321	validation_1-rmse:52.47943
[499]	validation_0-rmse:21.93978	validation_1-rmse:30.73111
Fold: 4/5 score = 943.88348
[0]	validation_0-rmse:50.99650	validation_1-rmse:50.67068
[449]	validation_0-rmse:22.42883	validation_1-rmse:29.86283
Fold: 5/5 score = 890.43725


In [12]:
for idx, model in enumerate(models):
  model.save_model(f"./model/xgboost_MLM_fold_{idx}.model")

In [ ]:
# importance = models[-1].get_booster().get_score(importance_type="weight")
# tuples = [(k, importance[k]) for k in importance]
# tuples.sort(key=lambda x: x[1])

# label, values = zip(*tuples)

# plt.rcParams["font.family"] = "MalGun Gothic"
# plt.figure(figsize=(10, 12))
# plt.barh(label[:100], values[:100])
# plt.show()

In [13]:
submit = pd.read_csv("./xgboost_optuna_v1.csv")
submit["MLM"] = np.mean(y_tests, axis=0)
submit.to_csv("xgboost_optuna_v1.csv", index=False)